<a href="https://colab.research.google.com/github/zronyj/fisicoquimica/blob/master/Hartree_Fock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¡Te volvemos a dar la bienvenida a otro cuaderno interactivo de Colaboratory!

Esta es la última práctica del curso. En esta ocasión vamos a conectar todo lo que hemos visto sobre mecánica cuántica con química. Para ello vamos a recordar algunas cosas, definir unas nuevas y poner en práctica un algoritmo que nos permitirá encontrar la energía de una molécula.

Vamos a comenzar importando las librerías necesarias.

In [0]:
%matplotlib inline
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

## Bases para H y He
Ahora, recordando lo aprendido con el átomo de hidrógeno, vamos a comenzar definiendo las bases para el hidrógeno y el helio con 2, 3 y 6 Gaussianas.

In [0]:
# Sets de bases
GTO = {"H":{}, "He":{}}

# Coeficiente exponencial Alfa para Hidrogeno
GTO["H"]["a"] = {2:[1.309756377, 0.233135974],
                 3:[3.42525091, 0.62391373, 0.16885540], 
                 6:[35.52322122, 6.513143725, 1.822142904,
                    0.625955266, 0.243076747, 0.100112428]}

# Coeficiente C para Hidrogeno
GTO["H"]["c"] = {2:[0.430128498, 0.678913531],
                 3:[0.15432897, 0.53532814, 0.44463454],
                 6:[0.00916359628, 0.04936149294, 0.16853830490,
                    0.37056279970, 0.41649152980, 0.13033408410]}

# Coeficiente exponencial Alfa para Helio
GTO["He"]["a"] = {2:[2.4328790, 0.4330510],
                  3:[6.36242139, 1.15892300, 0.31364979],
                  6:[65.98456824, 12.09819836, 3.384639924,
                     1.162715163, 0.451516322, 0.185959356]}

# Coeficiente C para Helio
GTO["He"]["c"] = {2:[0.430128498, 0.678913531],
                  3:[0.15432897, 0.53532814, 0.44463454],
                  6:[0.00916359628, 0.04936149294, 0.16853830490,
                     0.37056279970, 0.41649152980, 0.13033408410]}

Ahora, debemos comprender un par de conceptos importantes:
* Los orbitales de un átomo se ven afectados al ser estos parte de una molécula.
* La manera en que estos orbitales se ven afectados se puede entender como los coeficientes de una combinación lineal; cada vector es una función de onda.
* En base al método variacional, la cantidad a minimizar es la energía de la molécula.

Comencemos a ver esto de manera matemática antes de llegar a la parte computacional.

## Método Variacional
Vamos a asumir que la función de onda de una molécula es una función $\widetilde{\Phi}$. Entonces, la energía de la molécula podemos representarla como $E [ \widetilde{\Phi} ]$. En este caso, vamos a decir que $E$ es un funcional de $\widetilde{\Phi}$, pues se puede entender como la función de una función. Entonces, $E$ lo definimos como:

$$E [ \widetilde{\Phi} ] = \frac{\langle \widetilde{\Phi} | \hat{H} | \widetilde{\Phi} \rangle}{\langle \widetilde{\Phi} | \widetilde{\Phi} \rangle} \rightarrow E_{0}$$

En donde $\hat{H}$ es el operador Hamiltoniano, y lo que se busca es encontrar una función de onda que lleve a la energía mínima posible.

Entonces, lo primero que haremos será comenzar definiendo a $\widetilde{\Phi}$ como una función lineal de prueba.

$$| \widetilde{\Phi} \rangle = \sum_{i=1}^{N} c_i | \Psi_i \rangle$$

Esta función debe cumplir con lo planteado en el método variacional; eso es, buscar que la energía sea mínima.

$$E [ \widetilde{\Phi} ] = \langle \widetilde{\Phi} | \hat{H} | \widetilde{\Phi} \rangle = \sum_{ij} c_{i}^{*} c_{j} \langle \Psi_i | H | \Psi_j \rangle$$

En donde $H$ es una matriz cuyos elementos son el hamiltoniano para cada función de onda. A continuación, siguiendo con el método variacional, revisaremos que las funciones de onda estén normalizadas.

$$\langle \widetilde{\Phi} | \widetilde{\Phi} \rangle - 1 = \sum_{ij} c_{i}^{*} c_{j} \langle \Psi_i | \Psi_j \rangle - 1 = 0$$

Ahora, vamos a minimizar el funcional $E$ con respecto de los coeficientes $c_{i}^{*}$ y $c_{j}$. El problema, claramente, es que es un problema de muchas dimensiones y tenemos una restricción: la normalización. Es por eso que procedemos con el método de multiplicadores de Lagrange.

$$L = \langle \widetilde{\Phi} | \hat{H} | \widetilde{\Phi} \rangle - E [ \langle \widetilde{\Phi} | \widetilde{\Phi} \rangle - 1 ]$$

La última expresión, ya evaluando la función de prueba se ve de la siguiente manera:

$$L = \sum_{ij} c_{i}^{*} c_{j} \langle \Psi_i | H | \Psi_j \rangle - E [ \sum_{ij} c_{i}^{*} c_{j} \langle \Psi_i | \Psi_j \rangle - 1 ]$$

Siguiendo el método de multiplicadores de Lagrange, ahora es momento de calcular la derivada de esto. Posteriormente, esa derivada se igualará a cero para encontrar los máximos y mínimos.

$$\partial L = \sum_{ij} \partial c_{i}^{*} c_{j} \langle \Psi_i | H | \Psi_j \rangle - E [ \sum_{ij} \partial c_{i}^{*} c_{j} \langle \Psi_i | \Psi_j \rangle ] +\\ \sum_{ij} c_{i}^{*} \partial c_{j} \langle \Psi_i | H | \Psi_j \rangle - E [ \sum_{ij} c_{i}^{*} \partial c_{j} \langle \Psi_i | \Psi_j \rangle ]$$

Después de rearreglar algunos términos y factorizar otros, llegamos a la siguiente expresión:

$$\partial L = \sum_{i} \partial c_{i}^{*} \left[ \sum_{j} c_{j} \langle \Psi_i | H | \Psi_j \rangle - E c_{j} \langle \Psi_i | \Psi_j \rangle \right] +\\ \sum_{j} \partial c_{j} \left[ \sum_{i} c_{i}^{*} \langle \Psi_i | H | \Psi_j \rangle - E c_{i}^{*} \langle \Psi_i | \Psi_j \rangle \right] = 0$$

Debemos recordar que $H$ es una matriz, $\Psi$ es un vector y que las combinaciones lineales se pueden ver como multiplicaciones de vectores (productos punto). Entonces, las siguientes expresiones pueden reescribirse como matrices:
* $\langle \Psi_i | H | \Psi_j \rangle = H_{ij}$
* $\langle \Psi_i | \Psi_j \rangle = S_{ij}$

Finalmente, si la segunda sumatoria de la anterior ecuación es compleja ($c_{i}^{*}$ es el conjugado complejo de $c_{i}$), vamos a ocuparnos nada más de la primera parte: nos interesa el resultado real.

$$\sum_{i} \partial c_{i}^{*} \left[ \sum_{j} H_{ij} c_{j} - E S_{ij} c_{j} \right] = 0$$

De esta ecuación queda que $\sum_{j} H_{ij} c_{j} - E S_{ij} c_{j} = 0$, pues la constante no puede ser cero. Y al contemplar la multiplicación de matrices y vectores que se está haciendo dentro del paréntesis, esta ecuación se re-escribe como:

$$ H c = E S c $$

En este punto es conveniente notar que de trabajarse con **bases** y no las funciones directamente, el vector $c$ ya no es un vector, sino una matriz $C$, pues se necesita minimizar los coeficientes para cada función de base del orbital atómico. En este caso, la ecuación se torna:

$$ H C = S C E $$

Un último detalle es que la matriz $S$ surge de que se asume que las funciones $| \Psi \rangle$ no son ortonormales, por lo que el producto de dos funciones de estas da origen a coeficientes que se le llamarán de traslape. Es por ello que la matriz se conoce como matriz de traslape.

## Hartree-Fock

En las ecuaciones anteriores la matriz $H$ es un Hamiltoniano, pero no se especificó *qué* Hamiltoniano. Cuando se toma en cuenta la energía cinética de los electrones, potencial de atracción hacia el núcleo y potencial de repulsión entre electrones (cada par de electrones) para todos los electrones en una molécula, aparecen dos términos en el re-arreglo de variables del Hamiltoniano. Estos se reconocen como el operador de Coulomb $J$ y el operador de intercambio $K$.

El primero se origina de la energía potencial de repulsión de cada par de electrones en el mismo orbital, mientras que el segundo se origina de la antisimetría de la función de onda; es un resultado de la determinante de Slater. Entonces, al tomar en cuenta el Hamiltoniano para 1 electrón, el operador de Coulomb y el operador de Intercambio ($H^{1\ elec} + J + K$) se obtiene un nuevo Hamiltoniano que ya contempla la interacción entre cada par de electrones en una molécula. Este nuevo Hamiltoniano se conoce como el operador de Fock.

$$F = H^{1\ elec} + J + K = H^{1\ elec} + G$$

Al sustituir esto en las ecuaciones anteriores, se obtiene lo siguiente:

$$ F C = S C E $$

Esto casi se parece a un nuevo problema de valores propios. Sin embargo, sabemos que si el operador de Fock toma en cuenta la repulsión entre electrones y el intercambio de los mismos (para eso se asume que sabemos *a priori* su función de onda), este operador es una función de $C$.

$$F = F \left( C \right)$$

Esto implica que el problema requiere de una solución iterativa; no es posible hacerlo de manera analítica. Sin embargo, antes de entrar en ese procedimiento, vamos a terminar de resolver la ecuación matricial. (Esto sería más fácil si $S$ fuera la matriz identidad.) Es por eso que vamos a transformar el problema para convertir a $S$ en la matriz identidad.

Partiendo de que $S_{ij} = \langle \psi_i | \psi_j \rangle$ y las funciones $\Psi$ no son ortonormales, podemos plantear que exista una matriz de coeficientes $X_{ij}$ que, multiplicados por cada función de onda, construyan un conjunto ortonormal.

$$ \psi_{i}' = \sum_j X_{ji} \psi_{j}$$

Esto llevaría a que el producto de las funciones ya no sea la matriz $S$, sino la matriz identidad.

$$ \langle \psi_{i}' | \psi_{j}' \rangle = X^{\dagger} S X = U$$

En donde $U$ es la matriz identidad. Esto implica que diagonalizar $S$ para encontrar $X$ podría simplificar el problema. Una forma de hacer esto es mediante *ortogonalización canónica*.

$$ X_{ij} = \frac{U_{ij}}{s^{1/2}_{j}}$$

Aquí $s^{1/2}$ es una matriz diagonal cuyas entradas son la raíz cuadrada de los valores propios de $S$.

Ahora vamos a definir que la inversa de $X$ multiplicada por la matriz de coeficientes da como resultado una nueva matriz $C' = X^{-1} C$. Re-arreglando, nos damos cuenta de que $C = X C'$ y aquí es donde el problema se puede resolver.

$$ F X C' = S X C' E $$

Multiplicando por $X^{\dagger}$ por el lado izquierdo, se obtiene:

$$ \left( X^{\dagger} F X \right) C' = \left( X^{\dagger} S X \right) C' E $$

Definiendo una nueva forma del operador $F$ a partir de la ecuación anterior $F' = X^{\dagger} F X$, y recordando cómo se convertía $S$ en la matriz identidad, llegamos a la expresión:

$$ F' C' = C' E $$

Este problema ya se puede resolver al diagonalizar $F'$. Para obtener $C$, solo debemos recordar cómo definimos $C'$.

## Algoritmo de Hartree-Fock

Ahora que ya hemos comprendido cómo llegar a estas ecuaciones, vamos a proceder a incluir el código.

### Diagonalización
Lo primero que necesitamos son funciones para poder diagonalizar matrices de las maneras antes mencionadas.

In [0]:
def diagon(m=np.matrix([[1,-0.5],[-2,1.5]])):
    m_eigenval, m_eigenvec = np.linalg.eig(m)
    mvp = np.matrix(np.diag([1/i**0.5 for i in m_eigenval.tolist()]),
                    dtype=np.float64)
    return (m_eigenvec * mvp)[::-1]

def diagon2(m=np.matrix([[1,-0.5],[-2,1.5]])):
    if abs(m[0,0] - m[1,1]) < 1E-8:
        temp = np.pi/4
    elif (abs(m[0,1]) < 1E-8) or (abs(m[1,0]) < 1E-8):
        temp = np.pi/4
    else:
        temp = 0.5 * np.arctan( 2*m[0,1] / (m[0,0] - m[1,1]) )
    ed = np.cos(temp)
    ec = np.sin(temp)
    return np.matrix( [[ed,ec],[ec,-ed]] )

### Hamiltoniano y Matriz de Traslape
Ahora procederemos a definir el Hamiltoniano para moléculas diatómicas y su correspondiente matriz de traslape. Es importante notar que cada entrada de cada matriz debe de ser construida independientemente. Además, el Hamiltoniano se compone de una parte de energía potencial y otra de energía cinética.

In [0]:
# Elementos individuales de la matriz de traslape
def s (b1 = GTO["H"], b2 = GTO["H"], r = 0, b = 3):
    suv = 0
    for i in range(b-1,-1,-1):
        for j in range(b-1,-1,-1):
            du = (2*b1["a"][b][i]/np.pi)**0.75 * b1["c"][b][i]
            dv = (2*b2["a"][b][j]/np.pi)**0.75 * b2["c"][b][j]
            Sa = b1["a"][b][i] + b2["a"][b][j]
            suv += du * dv * (np.pi/Sa)**1.5 * \
            np.exp( -((b1["a"][b][i] * b2["a"][b][j])/Sa) * r**2 )
    return suv

# Matriz de traslape
def S(R = [0,1.4632], b1 = GTO["H"], b2 = GTO["H"], b = 3):
    MS = [[R[0]-R[0],R[1]-R[0]],
          [R[0]-R[1],R[1]-R[1]]]
    for i in range(2):
        for j in range(2):
            MS[i][j] = s(b1=b1, b2=b2, r=MS[i][j], b=b)
    return np.matrix(MS, dtype=np.float64)

In [0]:
# Elementos individuales de la matriz de energia cinetica
def t(b1 = GTO["H"], b2 = GTO["H"], r = 0, b = 3):
    tuv = 0
    for i in range(b-1,-1,-1):
        for j in range(b-1,-1,-1):
            du = (2*b1["a"][b][i]/np.pi)**0.75 * b1["c"][b][i]
            dv = (2*b2["a"][b][j]/np.pi)**0.75 * b2["c"][b][j]
            Sa = b1["a"][b][i] + b2["a"][b][j]
            Ma = b1["a"][b][i] * b2["a"][b][j]
            tuv += du * dv * (Ma/Sa) * (3 - 2*Ma/Sa * r**2) * \
            (np.pi/Sa)**1.5 * np.exp( -((Ma/Sa) * r**2 ))
    return tuv

# Matriz de energia cinetica
def T(R=[0,1.4632], b1 = GTO["H"], b2 = GTO["H"], b = 3):
    MT = [[R[0]-R[0],R[1]-R[0]],
          [R[0]-R[1],R[1]-R[1]]]
    for i in range(2):
        for j in range(2):
            MT[i][j] = t(b1=b1, b2=b2, r=MT[i][j], b=b)
    return np.matrix(MT, dtype=np.float64)

In [0]:
# Elementos individuales de la matriz de energia potencial
def v (b1 = GTO["H"], b2 = GTO["H"], r=1.4632, Rp=[0,0,1,1], Z=[1,1], b = 3):
    vuv = 0
    for k in range(len(Z)):
        R = [l*r for l in Rp[2*k:2*k+2]]
        for i in range(b-1,-1,-1):
            for j in range(b-1,-1,-1):
                du = (2*b1["a"][b][i]/np.pi)**0.75 * b1["c"][b][i]
                dv = (2*b2["a"][b][j]/np.pi)**0.75 * b2["c"][b][j]
                Sa = b1["a"][b][i] + b2["a"][b][j]
                Rr = (b1["a"][b][i] * R[0] + b2["a"][b][j] * R[1]) / Sa
                Ruv = abs(R[1]-R[0])
                Rur = (Rr - Ruv)**2
                t = Sa*Rur
                if t < 1E-5:
                    F = 1 - t
                else:
                    F = 0.5*(np.pi/t)**0.5 * erf(t**0.5)
                temp = (-2*du*dv*np.pi*Z[k]/Sa) * F * \
                np.exp( -((b1["a"][b][i] * b2["a"][b][j])/Sa * Ruv**2 ))
                vuv += N(temp)
    return vuv

# Matriz de energia potencial
def V (r = 1.4632, Z=[1,1], b1 = GTO["H"], b2 = GTO["H"], b = 3):
    MV = [[[0,0,1,1],[0,1,1,0]],
          [[1,0,0,1],[1,1,0,0]]]
    for i in range(2):
        for j in range(2):
            MV[i][j] = v(b1=b1, b2=b2, r=r, Rp=MV[i][j], Z=Z, b=b)
    return np.matrix(MV, dtype=np.float64)

In [0]:
# Hamiltoniano
def H (R=[0,1.4632], Z=[1,1], b1 = GTO["H"], b2 = GTO["H"], b = 3):
    th = T(R = R, b1 = b1, b2 = b2, b = 3)
    vh = V(r = R[1] - R[0], Z = Z, b1 = b1, b2 = b2, b = b)
    h = th + vh
    return h

### Operadores de Coulomb y de Intercambio
Ya contamos con $S$ y con $H^{1\ elec}$. Sin embargo, como habíamos visto, el operador de nuestro problema es el operador de Fock. Este toma en cuenta las energías de repulsión electrónica $J$ y de intercambio $K$. Estos operadores tienen como entradas integrales de dos electrones: es la tarea **más demandante** de este algoritmo.

In [0]:
# Integrales de dos electrones <ij|kl>
def ide (o = [0,0,0,0], R=[0,1], b1 = GTO["H"], b2 = GTO["H"], b = 3):
    res = 0
    base = [b1,b2]
    for i in range(b-1,-1,-1):
        for j in range(b-1,-1,-1):
            for k in range(b-1,-1,-1):
                for l in range(b-1,-1,-1):
                    s1 = base[o[0]]["a"][b][i] + base[o[1]]["a"][b][j]
                    s2 = base[o[2]]["a"][b][k] + base[o[3]]["a"][b][l]
                    s3 = s1 + s2
                    m1 = base[o[0]]["a"][b][i] * base[o[1]]["a"][b][j]
                    m2 = base[o[2]]["a"][b][k] * base[o[3]]["a"][b][l]
                    index = [i,j,k,l]
                    d = [(2*base[o[m]]["a"][b][index[m]]/np.pi)**0.75 * \
                         base[o[m]]["c"][b][index[m]] for m in range(4)]
                    contrac = np.prod(d)
                    R1 = R[o[1]] - R[o[0]]
                    R2 = R[o[3]] - R[o[2]]
                    Rr = (base[o[0]]["a"][b][i] * R[o[0]] + \
                          base[o[1]]["a"][b][j] * R[o[1]]) / s1
                    Rs = (base[o[2]]["a"][b][k] * R[o[2]] + \
                          base[o[3]]["a"][b][l] * R[o[3]]) / s2
                    Rsr = (Rs - Rr)**2
                    piterm = 2*np.pi**2.5 / (s1*s2*s3**0.5)
                    expterm = np.exp(-m1/s1 * R1**2 - m2/s2 * R2**2)
                    t = s1 * s2 / s3 * Rsr
                    if t < 1E-5:
                        F = 1 - t
                    else:
                        F = 0.5 * (np.pi/t)**0.5 * erf(t**0.5)
                    res += contrac * piterm * expterm * F
    return N(res)

In [0]:
# Matriz de dos electrones
def G (r=1.4632, p=np.matrix([[0,0],[0,0]], dtype=np.float64), b1=GTO["H"],
       b2=GTO["H"], b=3):
    g = [[0,0],[0,0]]
    for u in range(2):
        for v in range(2):
            for w in range(2):
                for x in range(2):
                    # Operador de Coulomb
                    j = ide(o=[u,v,w,x], R=[0,r], b1=b1, b2=b2, b=b)
                    # Operador de intercambio
                    k = ide(o=[u,x,w,v], R=[0,r], b1=b1, b2=b2, b=b)
                    g[u][v] += p[w,x] * ( j - 0.5 * k )
    return np.matrix(g, dtype=np.float64)

### Matriz de densidad de carga (P)
Para facilitar muchas cosas dentro de este algoritmo, y para poder extraer otro tipo de información a partir de las funciones de onda, se define la matriz de densidad de carga (o de orden de enlace) $P$. Esta se construye a partir de la matriz de coeficientes $C$.

$$P_{uv} = 2 \sum_{a}^{N/2} C_{ua}^{*} C_{va}$$

En donde $a$ indica el orbital molecular que se está utilizando y $u$ y $v$ son los índices de los coeficientes de cada orbital atómico. Esta matriz nos será útil en el algoritmo de Hartree-Fock para muchas cosas. Sin embargo, no debemos olvidar que $P$ depende de $C$.

In [0]:
def P (C=np.matrix([[0,0],[0,0]], dtype=np.float64)):
    p = [[C[0,0]*C[0,0], C[0,0]*C[1,0]],
         [C[1,0]*C[0,0], C[1,0]*C[1,0]]]
    return 2*np.matrix(p, dtype=np.float64)

### Método del Campo Autoconsistente (Self Consistent Field method [SCF])

Ya habiendo definido todas las matrices, lo que queda es ponerlas a funcionar según lo que vimos con anterioridad. Cada paso se evidencia en los comentarios.

In [0]:
def SCF (r = 1.4632, Z=[1,1], b1 = GTO["H"], b2 = GTO["H"], b = 3, vbs=False):
    R = [0, r]
    if vbs: print("*) Generando matriz de traslape S.")
    s_scf = S(R=R, b1=b1, b2=b2, b=b)
    if vbs: print("\n*) Generando hamiltoniano H.")
    h_scf = H(R=R, Z=Z, b1=b1, b2=b2, b=b)
        
    # Diagonalizar matriz S y hallar matriz X
    if vbs: print("\n*) Diagonalizando matriz S y hallando matriz diagonal X.")
    X = diagon(m=s_scf)
    Xa = X.getH()
    
    # Estimar matriz de densidad P
    if vbs: print("\n*) Creando matriz de densidad P.")
    p_scf = np.matrix([[0,0],[0,0]], dtype=np.float64)
    
    # Comenzar proceso iterativo
    if vbs: print("\n*) Comenzando con el SCF.")
    for iteracion in range(50):
        # Construir matriz de Fock F
        # F = H + G
        if vbs: print("\n**) Generando la matriz de Fock: calculando \
integrales de dos electrones.")
        g_scf = G(r=r, p=p_scf, b1=b1, b2=b2, b=b)
        f_scf = h_scf + g_scf
        
        # Construir matriz F'
        # F' = X_adj * F * X
        if vbs: print("**) Cambiando la base de F.")
        f_tra = Xa * f_scf * X
        
        # Diagonalizar matriz F y constuir matriz C'
        if vbs: print("**) Diagonalizando F' y generando C'.")
        c_tra = diagon2(m=f_tra)
        
        # Construir matriz C
        # C = X * C'
        if vbs: print("**) Construyendo matriz de coeficientes C.")
        c_scf = X * c_tra
        
        # Construir matriz P a partir de matriz C
        if vbs: print("**) Recalculando matriz de densidad P.")
        p_temp = P(C=c_scf)
        
        print("\nConcluida la " + str(iteracion + 1) + ". iteracion.\n")
        
        # Revisar convergencia
        if np.linalg.norm(p_temp - p_scf) < 1E-4:
            print("\n\n-->El campo autoconsistente SI ha convergido!")
            return {"S":s_scf,"H":h_scf,"X": X,"F":f_scf,"C":c_scf,"P":p_temp}
        else:
            p_scf = p_temp
    print("\n\n-->El campo autoconsistente NO ha convergido!\nRevisar supuestos.")
    return {"S":s_scf,"H":h_scf,"X": X,"F":f_scf,"C":c_scf,"P":p_temp}

Como podemos notar, lo que hace el SCF es realizar una serie de cálculos para determinar $C$, de allí calcular $P$ y volver a comenzar el algoritmo si es que este no ha convergido. La matriz $P$ se utiliza dentro del cálculo de la matriz de dos electrones $G$. Cuando el algoritmo termina de iterar (tiene un límite de 50 iteraciones), este devuelve el valor de todas las matrices encontradas. Estas las interpretaremos posteriormente.

Por ahora, vamos a correr el método de Hartree-Fock (o SCF) por primera vez.

In [0]:
# Vamos a calcular una molécula de hidrogeno H2
# Para ello la distancia interatomica sera: r = 1.4632
# La carga del nucleo de ambos atomos sera: Z = [1,1]
# Las bases que vamos a utilizar son las del hidrogeno: b1 = GTO["H"], b2 = GTO["H"]
# El numero de bases a utilizar esta vez sera: b = 3
# Vamos a pedir a la funcion que nos explique que esta pasando: vbs = True

# Para correr el SCF solo lo llamamos y almacenamos los resultados
resultados = SCF(r = 1.4632, Z=[1,1], b1 = GTO["H"], b2 = GTO["H"], b = 3, vbs = True)

## Resultados

Para comprender mejor lo que acabamos de calcular, recordemos que lo que buscábamos eran los coeficientes de las funciones de onda que hicieran mínima la energía de la molécula. Primero, vamos a ver los coeficientes.

In [0]:
print resultados["C"]

Esto no es muy revelador, pero al menos notamos que obtuvimos un resultado: existen coeficientes! Ahora comencemos a calcular algo más interesante con la demás información.

### Energía de cada orbital
La energía de cada orbital es la diagonal de $F'$. Este resultado no lo obtuvimos, así que lo debemos calcular.

In [0]:
def ener_orbs(x, f):
    return np.diag(x.getH() * f * x)

In [0]:
print ener_orbs(resultados["X"], resultados["F"])

De aquí no solo obtuvimos dos valores de energía, sino que podemos comenzar a entender que existen dos orbitales en la molécula de hidrógeno: uno de alta energía y uno de baja energía.

### Energía electrónica
La energía electrónica de la molécula se puede calcular a partir del operador de Fock. El procedimiento matemático implica descomponer el operador en un Hamiltoniano, la matriz de densidad de carga y la matriz de Fock propiamente.

In [0]:
def ener_elec(p, h, f):
    m = 0.5 * p.reshape(1,4) * (h + f).reshape(1,4).T
    return m[0,0]

In [0]:
print ener_elec(resultados["P"], resultados["H"], resultados["F"])

Este último resultado no contempla la energía potencial de repulsión entre los núcleos atómicos. Pero para fines de rupturas y formaciones de enlaces, esta energía es la más importante.

### Energía total
Finalmente, si deseamos la energía interna total de la molécula, solo incluimos la energía de repulsión nuclear.

In [0]:
def ener_tot(r=1.4632, Z=[1,1], elec=0):
    nuc = Z[0] * Z[1] / r
    return nuc + elec

In [0]:
electronica = ener_elec(resultados["P"], resultados["H"], resultados["F"])
print ener_tot(elec = electronica)

Con esto ya habríamos logrado obtener un resultado satisfactorio si lo que estuviéramos haciendo fuera un estudio termodinámico. Sin embargo, usualmente es mejor *ver* la molécula.

## Representaciones
Lo primero que vamos a visualizar son los orbitales en 1D. Así sabremos dónde encontrar al electrón.

Comenzamos definiendo la función de onda.

In [0]:
a, r, R = symbols("alpha r R")

phi = (2 * a / pi)**(3./4) * exp(-a * (r - R)**2)

def Phi (rn, Ru, base = GTO["H"], b = 3):
    wf = 0
    for i in range(b):
        wf += base["c"][b][i] * \
        phi.subs([(a, base["a"][b][i]), (r, rn), (R, Ru)])
    return wf

def Psi (r, R, base = GTO["H"], b = 3):
    wf = 0
    for i in range(b):
        a = base["a"][b][i]
        wf += base["c"][b][i] * (2 * a / np.pi)**0.75 * np.exp(-a * (r - R)**2)
    return wf

Luego creamos la función para graficar los orbitales.
En este punto es **muy importante** notar qué parámetros utiliza esta función, pues hasta aquí es donde se comienza a utilizar la información importante que habíamos obtenido del SCF.

In [0]:
def orbital (c, r = 1.4632, b1 = GTO["H"], b2 = GTO["H"], b = 3):
    dom = np.arange(-3.5,r+3.55,0.05)
    psi1 = [c[0,0] * Psi(x, 0, base=b1, b=b) + c[1,0] * \
            Psi(x, r, base=b2, b=b) for x in dom]
    psi2 = [c[0,1] * Psi(x, 0, base=b1, b=b) + c[1,1] * \
            Psi(x, r, base=b2, b=b) for x in dom]
    psi3 = [y**2 for y in psi1]
    psi4 = [y**2 for y in psi2]
    plt.title("Funciones de onda resultantes del calculo HF-SCF")
    plt.plot(dom, psi1, "r--", label="$\Psi_1$")
    plt.plot(dom, psi2, "b--", label="$\Psi_2$")
    plt.plot(dom, psi3, "g-", label="$\Psi_{1}^{2}$")
    plt.plot(dom, psi4, "k-", label="$\Psi_{2}^{2}$")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.grid(True)
    plt.show()

Ahora procedemos a graficar los orbitales.

In [0]:
orbital(resultados["C"], r = 1.4632, b1 = GTO["H"], b2 = GTO["H"], b = 3)

En este caso obtuvimos 4 funciones de onda en la gráfica:
1. La función de onda que junta más a los electrones.
2. La función de onda que aleja más a los electrones.
3. La función de onda que acerca más a los electrones, elevada al cuadrado (densidad electrónica).
4. La función de onda que aleja más a los electrones, elevada al cuadrado (densidad electrónica).

Si recordamos, habíamos obtenido **2** energías correspondientes a 2 orbitales para una misma molécula del átomo de hidrógeno. ¿Qué podrán significar 2 orbitales?

Pasemos a la gráfica en 2D.

In [0]:
def orbital2D (C, X, F, r = 1.4632, b1 = GTO["H"], b2 = GTO["H"], b = 3, delta=0.02):
    domx = np.arange(-3.5,r+3.5+delta,delta)
    domy = np.arange(-1.5-r/2,1.5+r/2+delta,delta)
    densmap1 = [[0]*len(domx)]*len(domy)
    densmap2 = [[0]*len(domx)]*len(domy)
    for y in range(len(domy)):
        q1 = Psi(domy[y], 0, base=b1, b=b)
        q2 = Psi(domy[y], 0, base=b2, b=b)
        for x in range(len(domx)):
            p1 = Psi(domx[x], 0, base=b1, b=b)
            p2 = Psi(domx[x], r, base=b2, b=b)
            densmap1[y][x] = (C[0,0]*p1*q1 + C[1,0]*p2*q2)**2
            densmap2[y][x] = (C[0,1]*p1*q1 + C[1,1]*p2*q2)**2
        densmap1[y] = tuple(densmap1[y])
        densmap2[y] = tuple(densmap2[y])
    dm1 = np.array(densmap1, dtype=np.float32)
    dm2 = np.array(densmap2, dtype=np.float32)
    energias = ener_orbs(X, F)
    fig = plt.figure()
    e1 = energias[0]
    a1 = fig.add_subplot(221)
    a1.title.set_text("Orbital 1\nE = " + str(e1))
    a1.imshow(dm1,cmap=cm.hot)
    e2 = energias[1]
    a2 = fig.add_subplot(224)
    a2.title.set_text("Orbital 2\nE = " + str(e2))
    a2.imshow(dm2,cmap=cm.hot)
    plt.show()

En este caso, notemos cómo esta gráfica utilizará más de nuestros resultados para construir una imagen. ¿Será que es necesaria más información que para la gráfica pasada? Revisemos bien el código para ver si notamos algo diferente.

Para mientras, vamos a graficar.

In [0]:
orbital2D(resultados["X"], resultados["C"], resultados["F"], r = 1.4632, b1 = GTO["H"], b2 = GTO["H"], b = 3)

Aquí tenemos nuevamente a los dos orbitales, esta vez acompañados de la energía de cada uno. Por si no lo habías descifrado aún, los dos orbitales que obtuvimos son el orbital de **enlace** y el orbital de **anti-enlace**. El primero tiene un valor de energía menor y muestra cómo los electrones se encuentran compartiendo el mismo espacio. El segundo muestra como los electrones no comparten el mismo espacio; más bien se repelen y no forman un enlace.

En este momento mereces una felicitación: es la primera vez que calculas un orbital molecular partiendo desde las bases. De manera teórica llegaste a construir el orbital molecular de la molécula de hidrógeno.

## Ejercicios

1. Vuelve a calcular la energía total de la molécula de hidrógeno, pero con 6 gaussianas en vez de 3. Vuelve a graficar los orbitales también. Anota tus hallazgos y compara los resultados entre las 2 bases.

2. Calcula la energía de la molécula de hidrógeno en ORCA. Vamos a especificarle al programa que deseamos un cálculo de tipo Hartree-Fock y con una base de 3 gaussianas. Luego incluiremos las coordenadas de los átomos despúes de especificar la carga de la molécula y la multiplicidad de los electrones (esto último solo se cambia si la molécula se encuentra excitada o con radicales libres). El documento de entrada se debe de ver entonces de la siguiente manera:
 
 `! HF STO-3G `
 
 `* xyz 0 1`
 
 ` H 0.0000 0.0000 0.0000`
 
 ` H 1.4632 0.0000 0.0000`
 
 `end`
 
 Cuando ya lo hayas ejecutado, revisa el documento de salida y compara esos resultados con los que habías obtenido aquí.
 
3. Calcula la energía total y construye las gráficas para la molécula HeH+ utilizando el SCF.
  * Utiliza 3 gaussianas en las bases
  * Recuerda cambiar las bases de H a He para uno de los átomos
  * Recuerda cambiar la carga nuclear también

4. Utiliza ORCA para calcular la energía según el método de **H**artree-**F**ock y una base **STO-3G** para *una* de las siguientes moleculas:
  * metano
  * metanol
  * cloroformo
  * cloruro de metileno
  * agua
  * amoníaco
  * floruro de hidrógeno
  * hidruro de litio
  * etano
  * etanol